# Vector similarity search through shared reference points

It was noted in [this blog post](https://softwaredoug.com/blog/2023/03/02/shared-dot-product.html) that if we know `u.A` and `v.A` we can estimate `u.v`. As an exercise, can we use that to prototype a vector similarity search?

## Why would this be useful?

* We can compress a large vector space to a much reduced few thousand reference vectors, called `refs` here
* We can index a set of vectors, `v`, by noting the most similar vectors to these `refs`, and storing the id and dot product `v.refs`
* We might put that index in a traditional index like a search system, and just let traditional text retrieval's similarity work to create cosine similarity between dense vectors

## Why would this be not very useful?

* We need to arbitrarily partition the space, so the reference points don't induce bias. However this means we are not sensitive to the actual 


## Import wikipedia sentences and vectors

Every 10 sentence of a wikipedia dump of sentences, totalling ~8m sentences/vectors. This is encoded with miniLM

```
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('all-MiniLM-L6-v2')
    model.encode(sentence)
```

In [1]:
import numpy as np
# All data - quite large for the entire set
# can be downlloaded from
# https://www.kaggle.com/datasets/softwaredoug/wikipedia-sentences-all-minilm-l6-v2
#
with open('wikisent2_all.npz', 'rb') as f:
    vects = np.load(f)
    vects = vects['arr_0']
    # vects = np.stack(vects)
    all_normed = (np.linalg.norm(vects, axis=1) > 0.99) & (np.linalg.norm(vects, axis=1) < 1.01)
    assert all_normed.all(), "Something is wrong - vectors are not normalized!"


with open('wikisent2.txt', 'rt') as f:
    sentences = f.readlines()
    
    
# # Smaller every 10 dataset

# with open('wikisent10.npz', 'rb') as f:
#     vects = np.load(f)
#     vects = np.stack(vects)
#     all_normed = (np.linalg.norm(vects, axis=1) > 0.99) & (np.linalg.norm(vects, axis=1) < 1.01)
#     assert all_normed.all(), "Something is wrong - vectors are not normalized!"

# with open('wikisent10.txt', 'rt') as f:
#     sentences = f.readlines()
    
vects.shape

(7871825, 384)

## Ground truth similarity

Take a dot product similarity to the query vector as the ground truth.

In [2]:
query = 1000000
query_vector = vects[query]
query_sentence = sentences[query]
query_sentence

'Cinnamon Cay is mostly covered with grass and cactus, and is located within the Virgin Islands National Park.\n'

In [3]:
nn = np.dot(vects, query_vector)
top_n = np.argpartition(-nn, 10)[:10]
top_n = top_n[nn[top_n].argsort()[::-1]]

for idx in top_n:
    print(idx, nn[idx], sentences[idx])

1000000 0.99999994 Cinnamon Cay is mostly covered with grass and cactus, and is located within the Virgin Islands National Park.

999999 0.84611005 Cinnamon Cay is a cay in the United States Virgin Islands, situated approximately 0.7 miles east of Trunk Cay in the Cinnamon Bay, and 100 yards from the shore at Cinnamon Bay Beach on Saint John island.

999996 0.6374875 Cinnamon Bay is a body of water and a beach on St. John island, within Virgin Islands National Park, in the United States Virgin Islands.

6343036 0.6151315 The mess is currently housed in the Cinnamon gardens area of Colombo.

5725906 0.5654203 The community was traditionally associated with the cultivation and management of cinnamon and are found mostly in Southern coastal areas of Sri Lanka.

3053785 0.5553695 It is found in the Cocos Islands, Moluccas, Lesser Sunda Islands, New Guinea, New Caledonia, Australia and the western Pacific.

2891321 0.55075276 It feeds on eucalyptus, especially swamp mahogany, and is found i

## Select a set of random vectors as reference points

We ensure we sample *randomly* otherwise the similarity below (summing similarities) won't work, as we'll be summing correlated similarities.

In [4]:
num_vectors = len(vects)

def centroid():
    """ Sample a unit vector from a sphere in N dimensions.
    It's actually important this is gaussian
    https://stackoverflow.com/questions/59954810/generate-random-points-on-10-dimensional-unit-sphere
    IE Don't do this
        projection = np.random.random_sample(size=num_dims)
        projection /= np.linalg.norm(projection)
    """
    num_dims = len(vects[0])
    projection = np.random.normal(size=num_dims)
    projection /= np.linalg.norm(projection)
    return projection   

centroid()

array([ 3.11857848e-02, -5.44647865e-02,  3.62523698e-04,  2.95326742e-02,
       -5.78769393e-02, -7.91969908e-03,  3.07337733e-02,  4.14794946e-02,
       -5.60921808e-02, -5.84231076e-03, -3.03641039e-02,  9.00863878e-02,
        3.40140830e-02,  2.63503545e-02, -1.00795820e-01,  3.90036975e-02,
       -7.67220630e-02, -5.09445686e-02,  1.63975372e-02,  8.20156256e-02,
       -9.47527599e-03, -5.44015110e-02,  1.07057850e-01,  7.64754614e-02,
       -1.70995573e-02, -3.54264228e-02,  1.39781900e-02, -9.59667981e-02,
        2.48580918e-02, -3.75828426e-02,  4.13216111e-02, -5.19822618e-02,
       -9.73467119e-03,  5.07475979e-02,  1.21104619e-02, -7.02523766e-02,
        4.40860425e-02, -8.25177776e-03, -3.68595383e-02, -2.36404814e-02,
        1.92492737e-02, -9.24268809e-02,  2.44768352e-02,  2.25758189e-02,
        2.02920451e-02,  7.23337102e-03,  4.41314485e-02, -3.62581152e-03,
        8.41175403e-02,  1.97563163e-02,  1.81072349e-01,  1.15561081e-01,
       -1.01793971e-01, -

## Most similar vectors to centroid

Get most similar vectors, with a specified floor in specificity. The top 10 here should correspond to the top 10 ground truth above.

In [8]:
def most_similar(centroid, floor):

    nn = np.dot(vects, centroid)
    idx_above_thresh = np.argwhere(nn >= floor)[: ,0]
    return list(zip(idx_above_thresh, nn[idx_above_thresh]))

nn_above_thresh = most_similar(query_vector, 0.01)
nn_above_thresh[:10]

[(0, 0.05252691),
 (2, 0.04499867),
 (5, 0.07958562),
 (6, 0.049663156),
 (7, 0.056482542),
 (14, 0.12559442),
 (15, 0.0227612),
 (16, 0.044335753),
 (19, 0.029843405),
 (20, 0.105600625)]

## Create a compressed index based on shared reference points

As mentioned in [this blog article](https://softwaredoug.com/blog/2023/03/02/shared-dot-product.html) we can use shared reference points between query and vector to estimate their similarity. Below we store

- A table of these reference vectors (`refs`) that can stand in for the full vector space
- A mapping of these `refs` -> a set of indexed vectors.

In [15]:
ref_neighbors = {}   # reference pts -> neighbors
ref_weights = {}     # (ref_ord, vect_ord) -> float
from time import perf_counter
from pyroaring import BitMap

num_refs = 4000
refs = np.zeros( (num_refs, vects.shape[1]) )

all_indexed_vectors = BitMap()

start = perf_counter()

for ref_ord in range(0, num_refs):
    specificity = 0.12
    
    center = centroid()    
    top_n = most_similar(center, specificity)
    
    refs[ref_ord, :] = center
    # idx = []
    bit_set = BitMap()
    for vector_ord, dot_prod in top_n:
        all_indexed_vectors.add(vector_ord)
        bit_set.add(vector_ord)
        ref_weights[(ref_ord,vector_ord)] = dot_prod
    ref_neighbors[ref_ord] = bit_set
    
    if ref_ord % 10 == 0:
        print(ref_ord, len(ref_neighbors[ref_ord]), len(all_indexed_vectors) / len(vects), perf_counter() - start)



0 77040 0.009786802933246103 8.181861333083361
10 107692 0.1339900213736967 38.28956733306404
20 173499 0.22692831713103379 68.17820320802275
30 67792 0.28654232023704795 98.14517087501008
40 44995 0.36756584400694886 128.37458304106258
50 212794 0.4404342068071889 158.73485320806503
60 66141 0.4978383030618694 188.78089770802762
70 27423 0.5365798909401568 218.9450416660402
80 79451 0.580765578503079 248.67051737499423
90 47747 0.6177246064286236 278.14855766599067
100 36658 0.669014364521569 308.11617233301513
110 15962 0.7033388064394216 338.0380136250751
120 195579 0.7469898022377276 368.2472629999975
130 25330 0.7620763927043601 398.0715541250538
140 134988 0.7834939165949446 428.5510214160895
150 58647 0.8042700390316095 458.83546000008937
160 229317 0.8206488330215674 489.0020875830669
170 49632 0.8307516998917024 519.2349730830174
180 37701 0.8460094069672535 549.425390999997
190 131710 0.8615008590765165 579.7196220000042
200 95865 0.872802431456492 610.0885791659821
210 29688

KeyboardInterrupt: 

In [ ]:
nn = np.dot(refs, query_vector)
nn[nn > 0]

# Search time!

Now when we search we go through the following steps, using just our reference points.

## Similarity to reference points

Compute similarity to `refs` from above...

In [ ]:
# Query vect -> refs similarity
nn = np.dot(refs, query_vector)

top_n_ref_points = np.argpartition(-nn, 10)[:10]
scored = nn[top_n_ref_points]

scored, top_n_ref_points

## Using reference points, As, estimate q.v

We have query vector `q`, which is similar to a set of reference points `A`, can we estimate `q.v`. We expect `q.v` to [approach `q.A*v.A` as we implement below](https://softwaredoug.com/blog/2023/03/02/shared-dot-product.html).

In [213]:
candidates = {}
cutoff = 0.0
for ref_ord, ref_score in zip(top_n_ref_points, scored):
    ref = refs[ref_ordinal]

    for vect_id, score in ref_neighbors[ref_ord]:
        # print(vect_id, score, score*ref_score)
        combined = score * ref_score
        if combined > cutoff:
            try:
                candidates[vect_id].append(combined)
            except KeyError:
                candidates[vect_id] = [combined]
            
list(candidates.items())[:10]

[(167639, [0.03348637198213025, 0.0215893165539961]),
 (181097, [0.03135131881147713]),
 (140960,
  [0.03068362407891349,
   0.01968200334702778,
   0.016470234480528256,
   0.021633394397366064]),
 (466857, [0.03044927361617932]),
 (451401, [0.030388548659453448, 0.01595256260519746]),
 (135071, [0.030258985871415626]),
 (608179, [0.029679072046431887, 0.01760330514900589]),
 (186829, [0.029606397626255852]),
 (227331, [0.029441571079467137]),
 (512583, [0.02939986434101547, 0.01587321875023174])]

## Sum the shared candidates

Should we sum the shared reference points?

Out of N reference points A0...AN we observe `u.A0...u.AN` and `v.0...vN`. We assume `u.v` would correlate to the dot product of these `u.A0*v.A0 + u.A1*v.A1 + ... + u.AN*v.AN`.

Note this only applies because we generate the reference points *randomly* introducing some bias in the reference points would create a case where many terms of the summation correlated heavily (the similarit yof `ref` `A0` and `A1` were so similar, that it was double counting). For example if `A0` and `A1` both occured towards the center of the data, we would be biased towards more general responses.

In [214]:
summed_candidates = {}
for vect_id, scored in candidates.items():
    summed_candidates[vect_id] = sum(scored)

In [215]:
results = summed_candidates.items()
results = sorted(results,
                 key=lambda scored: scored[1],
                 reverse=True)[:10]
# 21340
print("ZF -- ", query, sentences[query])
rank = -1
for idx, result in enumerate(results):
    print(idx, result, sentences[result[0]])

ZF --  100 10 in the UK Singles chart, however it was a bigger hit for Amazulu in 1986 from their album Amazulu.

0 (100, 0.2527549925590207) 10 in the UK Singles chart, however it was a bigger hit for Amazulu in 1986 from their album Amazulu.

1 (321056, 0.16217920551009507) It made 1 on the Dutch charts in 1986.

2 (323155, 0.15819849527952193) It peaked at #49 in the Australian singles chart.

3 (334214, 0.1571568908150194) It was a chart hit in Japan peaking at #40 and selling over 11,000 copies.

4 (321057, 0.154641193353647) It made #60 on the UK Singles Chart.

5 (323296, 0.1536705427431023) It peaked at number 86 on the UK Singles Chart.

6 (350317, 0.15308964160221297) It was released in July 1991 on the Festival Records label and spent nine weeks in the top 50 and peaked at No.34 on the Australian singles charts.

7 (323205, 0.15230754549290793) It peaked at number 14 in the United States, while peaking at number 11 in Canada.

8 (324573, 0.1521790017468438) It reached number

## Putting search together

Let's put the code above into one function.

In [245]:
def _search(query_vector, at=10):
    
    # query vect -> refs similarity
    nn = np.dot(refs, query_vector)

    top_n_ref_points = np.argpartition(-nn, 30)[:30]
    scored = nn[top_n_ref_points]

    # Candidates via our index
    candidates = {}
    cutoff = 0.0
    for ref_ord, ref_score in zip(top_n_ref_points, scored):
        ref = refs[ref_ordinal]

        for vect_id, score in ref_neighbors[ref_ord]:
            # print(vect_id, score, score*ref_score)
            combined = score * ref_score
            if combined > cutoff:
                try:
                    candidates[vect_id].append(combined)
                except KeyError:
                    candidates[vect_id] = [combined]
                    
    summed_candidates = {}
    for vect_id, scored in candidates.items():
        summed_candidates[vect_id] = sum(scored)
        
    results = summed_candidates.items()
    return sorted(results,
                  key=lambda scored: scored[1],
                  reverse=True)[:at]



from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

def search(query, at=10):
    query_vector = model.encode(query)
    return _search(query_vector, at)

def search_ground_truth(query, at=10):
    query_vector = model.encode(query)
    nn = np.dot(vects, query_vector)
    top_n = np.argpartition(-nn, at)[:at]
    top_n = top_n[nn[top_n].argsort()[::-1]]
    return sorted(zip(top_n, nn[top_n]),
                  key=lambda scored: scored[1],
                  reverse=True)
query = "best hamburger restaurants"
retrieved = set()
at=10
results = search(query, at=at)
retrieved = set()
for idx, result in enumerate(results[:10]):
    retrieved.add(result[0])
    print(idx, result, sentences[result[0]])

0 (646834, 0.3755975662324765) The only remaining structure of the original town is the Meers Store & Restaurant, which Food Network named as the best hamburger joint in Oklahoma & one of the best in the United States of America, largely due to its signature MeersBurger.

1 (668904, 0.28697694511992144) The restaurant is famous for its burgers and has been featured on Season 1 of the Travel Channel's Man vs. Food.

2 (644077, 0.27009526247083593) The Northbound side has a McDonald's (open 24 hours) and a KFC, with the Southbound side having a Burger King and a KFC.

3 (11317, 0.26373954450629955) Additionally, the chain also offers unique burgers in each city where its restaurants are located.

4 (77913, 0.25635830953768185) Bobby's Burger Palace features an array of burgers inspired by Chef Bobby Flay.

5 (697338, 0.2505928149242444) The stretch between Scott Street and Market Street is a popular restaurant area.

6 (324149, 0.24802131059412347) It provides information and reviews on 

In [246]:
results = search_ground_truth(query, at)
rank = -1
ground_truth = set()
for idx, result in enumerate(results):
    ground_truth.add(result[0])
    print(idx, result, sentences[result[0]])

0 (646834, 0.68789977) The only remaining structure of the original town is the Meers Store & Restaurant, which Food Network named as the best hamburger joint in Oklahoma & one of the best in the United States of America, largely due to its signature MeersBurger.

1 (307125, 0.59501857) It is known for its small, square hamburgers.

2 (11317, 0.59236324) Additionally, the chain also offers unique burgers in each city where its restaurants are located.

3 (653355, 0.58579004) The place serves American foods, such as burgers, hotdogs, chicken, etc.

4 (174575, 0.57885575) Hamburgers are grilled-to-order in full view of the customers and are served wrapped in paper on paper plates.

5 (331556, 0.57167995) Its tagline is The Last Great Hamburger Stand.

6 (331002, 0.5645297) Its restaurant network includes global chains like Burger King, Domino's Pizza, Kentucky Fried Chicken, Little Caesars Pizza, McDonald's, Pizza Hut, Pizza Pizza, Carl's Jr., Popeyes, Sampi, Papa John's and Subway.

7 (

In [271]:
test_queries = ["best buy electronics",
                "hamburger restaurant",
                "coffee shops in seattle",
                "ai chatbot",
                "endangered species of africa",
                "what is the capital of france?",
                "breaking bad main character",
                "what are underwear gnomes?",
                "funny reddit stories"]
at = 10
for query in test_queries:
    gt_results = search_ground_truth(query, at=at)
    retrieved_results = search(query, at=at)
    
    retrieved = set([result[0] for result in retrieved_results])
    ground_truth = set([result[0] for result in gt_results])
    
    print('-------')
    print(query, '--', len(ground_truth & retrieved) / len(ground_truth))
    print('GT - ', sentences[gt_results[0][0]].strip())
    print('RT - ', sentences[retrieved_results[0][0]].strip())

-------
best buy electronics -- 0.1
GT -  The device started shipping by August 2014, and eventually became widely available through Best Buy and Amazon, generating around $5 million in total retail sales.
RT -  ATR broke the Guinness Book of World Records for the Most Consumer Electronics Recycled in One Week (Multiple Locations) on April 25, 2015 during the OMPC.
-------
hamburger restaurant -- 0.3
GT -  It is known for its small, square hamburgers.
RT -  It is known for its small, square hamburgers.
-------
coffee shops in seattle -- 0.3
GT -  Hudsons Coffee is an Australian chain of coffee retailers.
RT -  Six of the eleven stores are located in Seattle (in the Fremont, Green Lake, Columbia City, View Ridge, and West Seattle neighborhoods).
-------
ai chatbot -- 0.3
GT -  He is currently CEO and co-founder of marketing-tech company Botworx.ai an artificial intelligence (AI) and natural language processing company that aims to help businesses interact with their customers via AI-pow

In [258]:
ref_lengths = [len(ref_neighbors[idx]) for idx in range(len(ref_neighbors))]
min(ref_lengths), max(ref_lengths)                   

(2483, 96611)

## Next steps

This is just a toy prototype of course, and would need to be evaluated for recall.

* Consider how you'd treat the reference points in a traditional search index (like Solr, Elasticsearch etc)
* Benchmark with more data (8m -> 80m wikipedia sentences)?
* Study the relationship of needed reference points to get good recall
* Test and ensure increasing reference points increases recall